# Figure 11

## Imports

In [ ]:
import dolfin
import matplotlib.pyplot as plt
import numpy
import sympy
import sys
import seaborn as sns

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

## Defining geometry and material parameters

In [ ]:
seeds_filename = "Fig11-seeds.dat"
mesh_filebasename = "Fig11-mesh"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI=0.,
    row=1,
    domain_y=domain_y,
    seeds_filename=seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename=mesh_filebasename,
    seeds_filename=seeds_filename,
    h=thickness,
    lcar=thickness/5,
    domain_x=domain_x,
    domain_y=domain_y,
    shift_y=0.,
    remove_seeds=True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

dV = dolfin.Measure("dx", domain=mesh)
coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

V0 = (xmax-xmin) * (ymax-ymin)
bbox = [xmin, xmax, ymin, ymax]


In [ ]:
E_  = 1.
nu_ = 0.49
mat_params = {"model":"CGNH", "parameters":{"E":E_, "nu":nu_}}

### Homogenized parameters

In [ ]:
homogenization_problem = dmech.HomogenizationProblem(
    dim=2,
    mesh=mesh,
    mat_params=mat_params["parameters"],
    vol=V0,
    bbox=bbox)
mu_tilde, lmbda_tilde = homogenization_problem.get_lambda_and_mu()

lmbda = sympy.Symbol('lambda')
mu    = sympy.Symbol('mu')

subs_dict        = {}
subs_dict[lmbda] = lmbda_tilde
subs_dict[mu]    = mu_tilde

## Computing macroscopic model response

### Kinematics variables

In [ ]:
# Cauchy-Green dilatation tensor
C_macro = sympy.MatrixSymbol('C', 2, 2).as_explicit()
print ("C_macro:"); C_macro

C_inv_macro = sympy.Inverse(C_macro)
print ("C_inv_macro:"); C_inv_macro.doit()

# Invariants
I_C_macro   = sympy.trace(C_macro)
III_C_macro = sympy.det(C_macro)
J_macro     = sympy.sqrt(III_C_macro)
print (  "I_C:"); I_C_macro
print ("III_C:"); III_C_macro
print (    "J:"); J_macro

# Reduced invariants
I_bar_C_macro = sympy.root(J_macro, -sympy.Rational(3, 2)) * I_C_macro
print ("I_bar_C:"); I_bar_C_macro


### Coupled model

In [ ]:
pf = sympy.symbols('p_f')

# Neo-Hookean free energy potential
W_c =   (lmbda/4) * (J_macro**2 - 1 - 2*sympy.ln(J_macro)) \
      + (   mu/2) * (I_C_macro  - 2 - 2*sympy.ln(J_macro))
print ("W_c:"); W_c.simplify()

# Second Piola-Kirchhoff stress tensor
Sigma_c = 2*sympy.diff(W_c, C_macro) - pf * J_macro * C_macro.inv()

p_c = - (sympy.trace(Sigma_c.T * C_macro) + lmbda/2 * (J_macro**2 - 1))/3/J_macro

Sigma_d_c = Sigma_c + p_c * J_macro * C_inv_macro
# print ("Sigma_d_c:"); Sigma_d_c

### Decoupled model

In [ ]:
K = lmbda + 2*mu/3
G = mu

# Neo-Hookean free energy potential
W_d = (K/4) * (J_macro**2 - 1 - 2*sympy.ln(J_macro)) \
    + (G/2) * (J_macro**(-2/3)*(1+ I_C_macro) - 3)
print ("W_d:"); W_d.simplify()

# Second Piola-Kirchhoff stress tensor
Sigma_d = 2*sympy.diff(W_d, C_macro) - pf * J_macro * C_macro.inv()

p_d = - (sympy.trace(Sigma_d.T * C_macro) + J_macro**(-2/3)*(1 - (sympy.trace(C_macro) + 1)/3)*G + K/2 * (J_macro**2 - 1))/3/J_macro

Sigma_d_d = Sigma_d + p_d * J_macro * C_inv_macro
# print ("Sigma_d_d:"); Sigma_d_d

## Stress-strain responses

### Deviatoric deformation

In [ ]:
# Deformation gradient
beta = sympy.symbols('beta')
F = sympy.Matrix(
    [[beta, 0     ],\
     [0   , 1/beta]])
print ("F:"); F

# Determinant of deformation gradient (a.k.a. Jacobian)
J = F.det()
print ("J:"); J

# Right Cauchy-Green dilatation tensor
C = F.T * F
print ("C:"); C

C_inv = sympy.Inverse(C).doit()
print ("C_inv:"); C_inv

#### Coupled model

In [ ]:
# Second Piola-Kirchhoff stress tensor
Sigma = Sigma_c.subs(list(zip(C_macro, C))) # list & zip should not be needed, cf. https://github.com/sympy/sympy/issues/10589
print ("Sigma:"); Sigma

# Hydrostatic pressure
p_c_shear = p_c.subs(list(zip(C_macro, C))) # list & zip should not be needed, cf. https://github.com/sympy/sympy/issues/10589
print ("p:"); p_c_shear.simplify()

# Deviatoric stress tensor
Sigma_d_c = Sigma_d_c.subs(list(zip(C_macro, C))) # list & zip should not be needed, cf. https://github.com/sympy/sympy/issues/10589
print ("Sigma_d:"); Sigma_d.simplify()
Sigma_c_VM = sympy.sqrt(1.5 * sympy.trace(Sigma_d_c.T * Sigma_d_c))

#### Decoupled model

In [ ]:
# Second Piola-Kirchhoff stress tensor
Sigma = Sigma_d.subs(list(zip(C_macro, C))) # list & zip should not be needed, cf. https://github.com/sympy/sympy/issues/10589
print ("Sigma:"); Sigma

# Hydrostatic pressure
p_d_shear = p_d.subs(list(zip(C_macro, C))).doit() # list & zip should not be needed, cf. https://github.com/sympy/sympy/issues/10589
print ("p:"); p_d_shear.simplify()

# Deviatoric stress tensor
Sigma_d_d = Sigma_d_d.subs(list(zip(C_macro, C))) # list & zip should not be needed, cf. https://github.com/sympy/sympy/issues/10589
print ("Sigma_d:"); Sigma_d.simplify()
Sigma_d_VM = sympy.sqrt(1.5 * sympy.trace(Sigma_d_d.T * Sigma_d_d))

## Defining microscopic model

In [ ]:
def global_response(mesh, mat_params, eps_xx, eps_yy, pf, sigma_bar, foi_name):

    dim = 2
    bcs = "pbc"
    step_params = {"dt_ini":1e-1, "dt_min":1e-3}

    res_folder = sys.argv[0][:-3]
    res_basename  = sys.argv[0][:-3]

    res_basename = res_folder+"/"+res_basename
    verbose=1

    ################################################################### Mesh ###

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    if (dim==2):    
        V0 = (xmax-xmin) * (ymax-ymin)
        bbox = [xmin, xmax, ymin, ymax]
    if (dim==3):
        zmax = max(coord[:,2]); zmin = min(coord[:,2])
        V0 = (xmax-xmin) * (ymax-ymin) * (zmax-zmin)
        bbox = [xmin, xmax, ymin, ymax, zmin, zmax]

    ################################################## Subdomains & Measures ###

    tol = 1E-8
    xmin_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmin, tol=tol)
    xmax_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmax, tol=tol)
    ymin_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymin, tol=tol)
    ymax_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymax, tol=tol)
    if (dim==3): zmin_sd = dolfin.CompiledSubDomain("near(x[2], x0, tol) && on_boundary", x0=zmin, tol=tol)
    if (dim==3): zmax_sd = dolfin.CompiledSubDomain("near(x[2], x0, tol) && on_boundary", x0=zmax, tol=tol)

    xmin_id = 1
    xmax_id = 2
    ymin_id = 3
    ymax_id = 4
    if (dim==3): zmin_id = 5
    if (dim==3): zmax_id = 6

    boundaries_mf = dolfin.MeshFunction("size_t", mesh, mesh.topology().dim()-1)
    boundaries_mf.set_all(0)
    xmin_sd.mark(boundaries_mf, xmin_id)
    xmax_sd.mark(boundaries_mf, xmax_id)
    ymin_sd.mark(boundaries_mf, ymin_id)
    ymax_sd.mark(boundaries_mf, ymax_id)
    if (dim==3): zmin_sd.mark(boundaries_mf, zmin_id)
    if (dim==3): zmax_sd.mark(boundaries_mf, zmax_id)

    if (verbose):
        xdmf_file_boundaries = dolfin.XDMFFile(res_basename+"-boundaries.xdmf")
        xdmf_file_boundaries.write(boundaries_mf)
        xdmf_file_boundaries.close()

    ################################################################ Problem ###

    problem = dmech.MicroPoroHyperelasticityProblem(
            mesh=mesh,
            mesh_bbox=bbox,
            boundaries_mf=boundaries_mf,
            displacement_perturbation_degree=1,
            quadrature_degree=3,
            solid_behavior=mat_params,
            bcs=bcs)

    ################################################################ Loading ###

    Deltat = step_params.get("Deltat", 1.)
    dt_ini = step_params.get("dt_ini", 1.)
    dt_min = step_params.get("dt_min", 1.)
    dt_max = step_params.get("dt_max", 1.)
    k_step = problem.add_step(
        Deltat=Deltat,
        dt_ini=dt_ini,
        dt_min=dt_min,
        dt_max=dt_max)

    problem.add_surface_pressure_loading_operator(
        measure=problem.dS(0),
        P_ini=0., P_fin=pf,
        k_step=k_step)

    for k in range(dim):
        for l in range (dim):
            if (sigma_bar[k][l] is not None):
                problem.add_macroscopic_stress_component_constraint_operator(
                    i=k, j=l,
                    sigma_bar_ij_ini=0.0, sigma_bar_ij_fin=sigma_bar[k][l],
                    pf_ini=0.0, pf_fin=pf,
                    k_step=k_step)

    problem.add_macroscopic_stretch_component_penalty_operator(
        i=0, j=0,
        U_bar_ij_ini=0.0, U_bar_ij_fin=eps_xx,
        pen_val=1e9,
        k_step=k_step)
    problem.add_macroscopic_stretch_component_penalty_operator(
        i=1, j=1,
        U_bar_ij_ini=0.0, U_bar_ij_fin=eps_yy,
        pen_val=1e9,
        k_step=k_step)

    for operator in problem.operators:
        if hasattr(operator, "material"):
            material = operator.material
            break

    problem.add_foi(expr=material.p_hydro, fs=problem.sfoi_fs, name="p_hydro", update_type="project")
    problem.add_foi(expr=material.Sigma_VM, fs=problem.sfoi_fs, name="Sigma_VM", update_type="project")

    ################################################################# Solver ###

    solver = dmech.NonlinearSolver(
        problem=problem,
        parameters={
            "sol_tol":[1e-6]*len(problem.subsols),
            "n_iter_max":32},
        relax_type="constant",
        write_iter=0)

    integrator = dmech.TimeIntegrator(
        problem=problem,
        solver=solver,
        parameters={
            "n_iter_for_accel":4,
            "n_iter_for_decel":16,
            "accel_coeff":2,
            "decel_coeff":2},
        print_out=res_basename*verbose,
        print_sta=res_basename*verbose,
        write_qois=res_basename+"-qois",
        write_qois_limited_precision=1,
        write_sol=res_basename*verbose)

    success = integrator.integrate()
    assert (success),\
        "Integration failed. Aborting."

    integrator.close()

    ############################################################################

    U_bar = problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar
    v = J_bar * V0
    vs = dolfin.assemble(problem.kinematics.J * problem.dV)
    vf = v - vs

    for operator in problem.operators:
        if hasattr(operator, "material"):
            material = operator.material
            break

    data_dist = None
    for i in range(len(problem.fois)):
        if problem.fois[i].name == foi_name:
            data_dist = problem.fois[i].func.vector().get_local()

    return data_dist

In [ ]:
sigma_bar = [[None, 0.],
             [0., None]]

## Computing quantities frequency distribution from the microscopic model

In [ ]:
pf_ = 0.
beta_ = 1.25

data_dist_micro =  global_response(mesh=mesh, mat_params=mat_params, eps_xx=beta_-1, eps_yy=1/beta_-1, pf=pf_, sigma_bar=sigma_bar, foi_name='p_hydro')
data_macro = p_c_shear.subs(beta, beta_).subs(subs_dict).subs(pf, pf_)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel('Hydrostatic pressure (kPa)', fontsize=16)
plt.ylabel('Frequency distribution', fontsize=16)
sns.histplot(data_dist_micro, bins=60, kde=True, color='#084594', stat="density", kde_kws=dict(cut=3), alpha=0.4, edgecolor=(1, 1, 1, 0.4))
plt.axvline(x=data_macro, color='#99000D', label='axvline - full height')
plt.legend(['Microscopic model', 'Macroscopic model'])
plt.xlim(-1., + 1.)
plt.ylim( 0.,  10.)
plt.show()
plt.savefig('Fig11-p_hydro_density_P00.pdf', bbox_inches='tight')

In [ ]:
pf_ = 0.2
beta_ = 1.25

data_dist_micro =  global_response(mesh=mesh, mat_params=mat_params, eps_xx=beta_-1, eps_yy=1/beta_-1, pf=pf_, sigma_bar=sigma_bar, foi_name='p_hydro')
data_macro = p_c_shear.subs(beta, beta_).subs(subs_dict).subs(pf, pf_)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel('Hydrostatic pressure (kPa)', fontsize=16)
plt.ylabel('Frequency distribution', fontsize=16)
sns.histplot(data_dist_micro, bins=60, kde=True, color='#084594', stat="density", kde_kws=dict(cut=3), alpha=0.4, edgecolor=(1, 1, 1, 0.4))
plt.axvline(x=data_macro, color='#99000D', label='axvline - full height')
plt.legend(['Microscopic model', 'Macroscopic model'])
plt.xlim(-1., + 1.)
plt.ylim( 0.,  10.)
plt.show()
plt.savefig('Fig11-p_hydro_density_P02.pdf', bbox_inches='tight')

In [ ]:
pf_ = 0.
beta_ = 1.25

data_dist_micro =  global_response(mesh=mesh, mat_params=mat_params, eps_xx=beta_-1, eps_yy=1/beta_-1, pf=pf_, sigma_bar=sigma_bar, foi_name='Sigma_VM')
data_macro = Sigma_c_VM.subs(beta, beta_).subs(subs_dict).subs(pf, pf_).simplify()

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel('Von Mises stress (kPa)', fontsize=16)
plt.ylabel('Frequency distribution', fontsize=16)
sns.histplot(data_dist_micro, bins=60, kde=True, color='#084594', stat="density", kde_kws=dict(cut=3), alpha=0.4, edgecolor=(1, 1, 1, 0.4))
plt.axvline(x=data_macro, color='#99000D', label='axvline - full height')
plt.legend(['Microscopic model', 'Macroscopic model'])
plt.xlim(0., 0.8)
plt.ylim(0., 10.)
plt.show()
plt.savefig('Fig11-Sigma_VM_density_P00.pdf', bbox_inches='tight')

In [ ]:
pf_ = 0.2
beta_ = 1.25

data_dist_micro =  global_response(mesh=mesh, mat_params=mat_params, eps_xx=beta_-1, eps_yy=1/beta_-1, pf=pf_, sigma_bar=sigma_bar, foi_name='Sigma_VM')
data_macro = Sigma_c_VM.subs(beta, beta_).subs(subs_dict).subs(pf, pf_).simplify()

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)        
plt.xlabel('Von Mises stress (kPa)', fontsize=16)
plt.ylabel('Frequency distribution', fontsize=16)
sns.histplot(data_dist_micro, bins=60, kde=True, color='#084594', stat="density", kde_kws=dict(cut=3), alpha=0.4, edgecolor=(1, 1, 1, 0.4))
plt.axvline(x=data_macro, color='#99000D', label='axvline - full height')
plt.legend(['Microscopic model', 'Macroscopic model'])
plt.xlim(0., 0.8)
plt.ylim(0., 10.)
plt.show()
plt.savefig('Fig11-Sigma_VM_density_P02.pdf', bbox_inches='tight')